In [1]:
# Import all necessary libraries

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import scipy
import sklearn
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import seaborn as sns
import os
import math
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import plotly.tools as pt

from sklearn.cluster import KMeans
from  scipy.stats import skew, kurtosis, shapiro
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import median_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# To check the list of all the CSV files
for root, dirs, files in os.walk("hhblock_dataset"):
    for filename in files:
        print(filename)

block_71.csv
block_65.csv
block_59.csv
block_58.csv
block_64.csv
block_70.csv
block_99.csv
block_66.csv
block_72.csv
block_73.csv
block_67.csv
block_98.csv
block_88.csv
block_63.csv
block_77.csv
block_76.csv
block_62.csv
block_89.csv
block_48.csv
block_74.csv
block_60.csv
block_61.csv
block_75.csv
block_49.csv
block_108.csv
block_12.csv
block_4.csv
block_5.csv
block_13.csv
block_109.csv
block_11.csv
block_39.csv
block_7.csv
block_6.csv
block_38.csv
block_10.csv
block_28.csv
block_14.csv
block_2.csv
block_3.csv
block_15.csv
block_29.csv
block_17.csv
block_1.csv
block_0.csv
block_16.csv
block_101.csv
block_33.csv
block_27.csv
block_26.csv
block_32.csv
block_100.csv
block_102.csv
block_24.csv
block_30.csv
block_18.csv
block_19.csv
block_31.csv
block_25.csv
block_103.csv
block_107.csv
block_21.csv
block_35.csv
block_34.csv
block_20.csv
block_106.csv
block_110.csv
block_104.csv
block_36.csv
block_22.csv
block_8.csv
block_9.csv
block_23.csv
block_37.csv
block_105.csv
block_111.csv
block_93.c

In [3]:
def changeDateFormat(date): 
    date = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').strftime('%d-%m-%Y')
    return(date)

weather = pd.read_csv('weather_daily_darksky.csv')
dtx = []
for t in weather['time']:
    dtx.append(changeDateFormat(t))
weather['date'] = pd.DataFrame(dtx)

# selecting numeric variables
weather = weather[['temperatureMax', 'windBearing', 'dewPoint', 'cloudCover', 'windSpeed',
       'pressure', 'apparentTemperatureHigh', 'visibility', 'humidity',
       'apparentTemperatureLow', 'apparentTemperatureMax', 'uvIndex',
       'temperatureLow', 'temperatureMin', 'temperatureHigh',
       'apparentTemperatureMin', 'moonPhase','date']]
weather = weather.dropna()
scaler = MinMaxScaler()
weather_scaled = scaler.fit_transform(weather[['temperatureMax','humidity','windSpeed']])
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
score = [kmeans[i].fit(weather_scaled).score(weather_scaled) for i in range(len(kmeans))]
kmeans = KMeans(n_clusters=3, max_iter=600, algorithm = 'auto')
kmeans.fit(weather_scaled)
weather['weather_cluster'] = kmeans.labels_

holiday = pd.read_csv('uk_bank_holidays.csv')
dtx = []
for t in holiday['Bank holidays']:
    dtx.append(dt.datetime.strptime(t, '%Y-%m-%d').strftime('%d-%m-%Y'))
        
holiday['Bank holidays'] = pd.DataFrame(dtx)

weather_holiday = weather.merge(holiday, left_on = 'date',right_on = 'Bank holidays',how = 'left')
weather_holiday['holiday_ind'] = np.where(weather_holiday['Bank holidays'].isna(),0,1)

In [ ]:
#### List of all the values between 0-111 for the first dropdown to select Block ####
block_list = [*range(0, 112, 1)] 

############### Create the dataset based on user selection ####################
def reframe_dataset(block,household):
    hh_name = "hhblock_dataset/block_"+str(block)+".csv"
    df = pd.read_csv(hh_name)
    df_test = df[df['LCLid'] == str(household)]
    df_test['total_consumption'] = round(df_test.sum(axis=1),2)
    try:
        df_test['day_'] = pd.to_datetime(df_test['day'], format='%d-%m-%Y')
    except:
        df_test['day_'] = pd.to_datetime(df_test['day'], format='%Y-%m-%d')
    df_test['year'] = pd.DatetimeIndex(df_test['day']).year
    df_test['month'] = pd.DatetimeIndex(df_test['day']).month
    df_test['date'] = pd.DatetimeIndex(df_test['day']).day
    df_test = df_test[['day','total_consumption','date','month','year']]
    return df_test

############### convert an array of values into a dataset matrix ############
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

################ MAPE value function ######################
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

############### Evaluation result ##################
def evaluate_ts(testY,testPredict):
    MAE = round(median_absolute_error(testY,testPredict),4)
    MAPE = round(mean_absolute_percentage_error(testY,testPredict),4)
    MSE = round(mean_squared_error(testY,testPredict),4)
    RMSE = round(math.sqrt(MSE),4)
    return MAE,MAPE,MSE,RMSE

############## Merge Weather dataset with temperature ###############
def merge_dataset(block,household):
    df = reframe_dataset(block,household)
    weather_energy = df.merge(weather_holiday,left_on='day',right_on='date')
    weather_energy.set_index(['day'],inplace=True)
    return weather_energy

########## Split dataset into train and test sets #####################
def split_to_train_test(dataset,train_percent):
    train, test = train_test_split(dataset, train_size = train_percent,shuffle=False)
    return train,test
    
########### SARIMA function to train & validate the model #####################
def model_SARIMA(block,household,train_percent):
    df = merge_dataset(block,household)
#     print(df)
    train, test = train_test_split(df, train_size = train_percent,shuffle=False)
    endog = train['total_consumption']
# exog = train[['weather_cluster','holiday_ind']]
    mod = sm.tsa.statespace.SARIMAX(endog=endog, order=(1,0,1),seasonal_order=(1,1, 0, 12))
    model_fit = mod.fit()
    model_fit.summary()
    
    # Prediction
    predict = model_fit.predict(start = len(train),end = len(train)+len(test)-1)
    testPredicted = predict.values
    
    
    fig_map, gr = plt.subplots()

    gr.plot(test['total_consumption'])
    gr.plot(testPredicted)
    
    pred_graph = pt.mpl_to_plotly(fig_map)

    MAE,MAPE,MSE,RMSE = evaluate_ts(test['total_consumption'],testPredicted)
    return MAE,MAPE,MSE,RMSE,pred_graph
    
########### MLP function to train & validate the model #####################
def MLP(block,household,train_percent,look_back):
    df_test = reframe_dataset(block,household)
    df_tc = df_test['total_consumption']
    df_tc = pd.DataFrame(df_tc)
    dataset = df_tc.values
    train_size = int(len(dataset) * train_percent)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,0], dataset[train_size:len(dataset),0]

    trainX, trainY = create_dataset(train, int(look_back))
    testX, testY = create_dataset(test, int(look_back))
    
    # create and fit Multilayer Perceptron model
    model = Sequential()
    model.add(Dense(12, input_dim=int(look_back), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=400, batch_size=2, verbose=2)


    # Estimate model performance
    trainScore = model.evaluate(trainX, trainY, verbose=0)
    testScore = model.evaluate(testX, testY, verbose=0)

    # generate predictions for training
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(dataset)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

    # shift test predictions for plotting
    testPredictPlot = np.empty_like(dataset)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    
    fig_map, gr = plt.subplots()

    gr.plot(dataset)
    gr.plot(trainPredictPlot)
    gr.plot(testPredictPlot)
    pred_graph = pt.mpl_to_plotly(fig_map)
    
    MAE, MAPE, MSE, RMSE = evaluate_ts(testY,testPredict)
    
    return MAE,MAPE,MSE,RMSE, pred_graph

############End of user defined functions###################

df = pd.DataFrame()
fig = go.Figure()

app = dash.Dash()

app.layout = html.Div(
    [
        html.Div([html.H1(children='Electricity Load Forecasting')],style={'width':'100%','textAlign': 'center'}),
        html.Div([
            html.H3('Inputs'),
            html.Label(["Select a Block",
                        dcc.Dropdown(
                            id='opt-block',
                            options=[{'label': s, 'value': s} for s in block_list],
                            value=0,
                            placeholder = 'Please select a block',multi=False),
                        html.Label(id='display-selected-block',style={'margin':'15px'}),],
                       style={'width': '30%', 'display': 'inline-block', 'margin':'10px'}),
            html.Label(["Select a Household",
                        dcc.Dropdown(
                            id='opt-household',
                            placeholder = 'Please select a Household'
                        ),
                        html.Label(id='display-selected-household',style={'margin':'15px'}
                                  ),
                       ],
                       style={'width': '30%', 'display': 'inline-block' ,'margin': '10px'}
                      ),
            html.Label(["Select train set size (%)",
                        dcc.Slider(
                            id='my-slider',
                            min=0,
                            max=1,
                            step=0.1,
                            marks={0:'0',0.1:'10',0.2:'20',0.3:'30',0.4:'40',0.5:'50',0.6:'60',0.7:'70',0.8:'80',0.9:'90',1:'100'},
                            value=0.8,
                        ),
                        html.Label(id='slider-output-container',style={'margin':'15px'}),
                       ],style={'width': '30%', 'display': 'inline-block' ,'margin':'10px'}),
            html.Br()
        ]),
        html.Div([
            html.H3('Output'),
            html.Label("View of selected dataset"),
            dash_table.DataTable(id='data_frame',
                                 columns=[{"name": i, "id": i} for i in df.columns],
                                 data=df.to_dict('records'),
                                 page_size=10,
                                 style_table = {'border': 'thin lightgrey solid'}),
            html.H4(id='display-split-tt',style={'margin':'15px'}),
            
            html.Div([
                html.Label([
                    html.Label([
                        html.H4('Data Exploration'),
                        dcc.Graph(id='vis-1',figure=fig),
                        html.Div(html.H4(id='display-results-vis1')),], 
                        style = {'width':'33%','display' : 'inline-block'}),
                    
                    html.Label([
                        html.H4('Prediction MLP'),
                        dcc.Graph(id='graph-1',figure=fig),
                        html.Div(html.H4(id='display-results-MLP')),],
                        style = {'width':'33%','display':'inline-block'}),
                   
                    
                    html.Label([
                        html.H4('Prediction SARIMA'),
                        dcc.Graph(id='graph-2',figure=fig),
                        html.Div(html.H4(id='display-results-SARIMA')),],
                        style = {'width':'33%','display':'inline-block'}),
                ]),      
            ]),
        ]),
    ])


@app.callback(
    dash.dependencies.Output('opt-household', 'options'),
    [dash.dependencies.Input('opt-block', 'value')]
)
def update_date_dropdown(name):
    hh_name = "hhblock_dataset/block_"+str(name)+".csv"
    df = pd.read_csv(hh_name)
    hh_list = df.LCLid.unique()
    return [{'label': i, 'value': i} for i in hh_list]

@app.callback(
    dash.dependencies.Output('display-selected-block', 'children'),
    [dash.dependencies.Input('opt-block', 'value')])
def set_display_block(selected_value):
    return 'Block selected: {}'.format(selected_value)

@app.callback(
    dash.dependencies.Output('display-selected-household', 'children'),
    [dash.dependencies.Input('opt-household', 'value')])
def set_display_household(selected_value):
    return 'Household selected: {}'.format(selected_value)

@app.callback(
    dash.dependencies.Output('slider-output-container', 'children'),
    [dash.dependencies.Input('my-slider', 'value')])
def update_output(value):
    return 'Train set portion {} %'.format(value * 100)


@app.callback(
    [dash.dependencies.Output('data_frame','data'),
    dash.dependencies.Output('data_frame','columns')],
    [dash.dependencies.Input('opt-block','value'),
        dash.dependencies.Input('opt-household','value')])
def selected_df(block,household):
    df_test = reframe_dataset(block,household)
    return (df_test.to_dict('records'), [{"name": i, "id": i} for i in df_test.columns])

@app.callback(
    dash.dependencies.Output('display-split-tt','children'),
    [dash.dependencies.Input('opt-block','value'),
    dash.dependencies.Input('opt-household','value'),
    dash.dependencies.Input('my-slider','value')])
def show_len_train_test(block,household,train_percent):
    df = reframe_dataset(block,household)
    train, test = split_to_train_test(df,train_percent)
    return 'Length of: Dataset: {}, Train-set: {}, Test-set: {}'.format(len(df),len(train),len(test))

@app.callback(
    [dash.dependencies.Output('display-results-MLP','children'),
    dash.dependencies.Output('graph-1','figure')],
    [dash.dependencies.Input('opt-block','value'),
    dash.dependencies.Input('opt-household','value'),
    dash.dependencies.Input('my-slider','value')])
def model_exec(block,household,train_percent,look_back=2):
    MAE, MAPE, MSE, RMSE, figure = MLP(block,household,train_percent,look_back)
#     print("Done here")
    return 'MLP Results: MAE: {}, MAPE: {}, MSE: {}, RMSE: {}'.format(MAE,MAPE,MSE,RMSE), figure
#     return figure
    
@app.callback(
    [dash.dependencies.Output('vis-1','figure'),
     dash.dependencies.Output('display-results-vis1','children')],
    [dash.dependencies.Input('opt-block','value'),
    dash.dependencies.Input('opt-household','value')])    
def data_distribution(block,household):
    dataset = reframe_dataset(block,household)
#     mean,std,skewness,ex_kurt = mean_std_skew(dataset)
    fig_map1, p = plt.subplots()
    sns.distplot(dataset['total_consumption'])
    mean = np.mean(dataset.total_consumption.values)
    std = np.std(dataset.total_consumption.values)
    figure = pt.mpl_to_plotly(fig_map1)
    return figure, 'Mean: {}, Standar Deviation: {}'.format(mean,std)

@app.callback(
    [dash.dependencies.Output('display-results-SARIMA','children'),
    dash.dependencies.Output('graph-2','figure')],
    [dash.dependencies.Input('opt-block','value'),
    dash.dependencies.Input('opt-household','value'),
    dash.dependencies.Input('my-slider','value')])
def model_exec(block,household,train_percent):
    MAE, MAPE, MSE, RMSE, figure = model_SARIMA(block,household,train_percent)
#     print("Done here")
    return 'SARIMA Results: MAE: {}, MAPE: {}, MSE: {}, RMSE: {}'.format(MAE,MAPE,MSE,RMSE), figure
#     return figure


# @app.callback(
#     dash.dependencies.Output('vis-2','figure'),
#     [dash.dependencies.Input('opt-block','value'),
#     dash.dependencies.Input('opt-household','value')]) 
# def data_viz_plots(block,household):
#     dataset = reframe_dataset(block,household)
#     fig_map2, p = plt.subplots()
#     fig_map2 = sns.jointplot(x=dataset["year"],y=dataset["total_consumption"],data=dataset,kind='scatter')
#     figure = pt.mpl_to_plotly(fig_map2)
#     return figure
    
if __name__ == '__main__':
    app.run_server(port=8001,debug=False)

Dash is running on http://127.0.0.1:8001/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8001/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2020 15:33:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 15:33:11] "GET /_dash-component-suites/dash_core_components/async-graph.v1_10_2m1595872686.js HTTP/1.1" 200 -
/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/seaborn/distributions.py:200: RuntimeWarning:

Mean of empty slice.

/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value enc

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

127.0.0.1 - - [01/Sep/2020 15:33:11] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

127.0.0.1 - - [01/Sep/2020 17:21:42] "POST /_dash-update-component HTTP/1.1" 500 -
/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

127.0.0.1 - - [01/Sep/2020 17:21:43] "POST /_dash-update-component HTTP/1.1" 200 -


Epoch 1/400
 - 1s - loss: 196.3249
Epoch 2/400
 - 0s - loss: 18.2393
Epoch 3/400
 - 0s - loss: 17.6805
Epoch 4/400
 - 0s - loss: 17.4249
Epoch 5/400
 - 0s - loss: 17.3740
Epoch 6/400
 - 0s - loss: 17.0915
Epoch 7/400
 - 0s - loss: 16.9934
Epoch 8/400
 - 0s - loss: 17.2164
Epoch 9/400
 - 0s - loss: 17.2078
Epoch 10/400
 - 0s - loss: 16.9463
Epoch 11/400
 - 0s - loss: 16.8686
Epoch 12/400
 - 0s - loss: 16.7491
Epoch 13/400
 - 0s - loss: 16.8784
Epoch 14/400
 - 0s - loss: 16.7452
Epoch 15/400
 - 0s - loss: 16.7224
Epoch 16/400
 - 0s - loss: 16.7240
Epoch 17/400
 - 0s - loss: 16.7972
Epoch 18/400
 - 0s - loss: 16.7749
Epoch 19/400
 - 0s - loss: 16.7681
Epoch 20/400
 - 0s - loss: 16.7577
Epoch 21/400
 - 0s - loss: 16.4063
Epoch 22/400
 - 0s - loss: 16.5750
Epoch 23/400
 - 0s - loss: 16.6640
Epoch 24/400
 - 0s - loss: 16.5394
Epoch 25/400
 - 0s - loss: 16.6160
Epoch 26/400
 - 0s - loss: 16.5054
Epoch 27/400
 - 0s - loss: 16.1996
Epoch 28/400
 - 0s - loss: 16.6088
Epoch 29/400
 - 0s - loss: 1

Epoch 232/400
 - 0s - loss: 14.4190
Epoch 233/400
 - 0s - loss: 14.4172
Epoch 234/400
 - 0s - loss: 14.5985
Epoch 235/400
 - 0s - loss: 14.5086
Epoch 236/400
 - 0s - loss: 14.4745
Epoch 237/400
 - 0s - loss: 14.3808
Epoch 238/400
 - 0s - loss: 14.5283
Epoch 239/400
 - 0s - loss: 14.6372
Epoch 240/400
 - 0s - loss: 14.4575
Epoch 241/400
 - 0s - loss: 14.4225
Epoch 242/400
 - 0s - loss: 14.5575
Epoch 243/400
 - 0s - loss: 14.4825
Epoch 244/400
 - 0s - loss: 14.6155
Epoch 245/400
 - 0s - loss: 14.3439
Epoch 246/400
 - 0s - loss: 14.4804
Epoch 247/400
 - 0s - loss: 14.4362
Epoch 248/400
 - 0s - loss: 14.5067
Epoch 249/400
 - 0s - loss: 14.3922
Epoch 250/400
 - 0s - loss: 14.4201
Epoch 251/400
 - 0s - loss: 14.4460
Epoch 252/400
 - 0s - loss: 14.4281
Epoch 253/400
 - 0s - loss: 14.3813
Epoch 254/400
 - 0s - loss: 14.4368
Epoch 255/400
 - 0s - loss: 14.5037
Epoch 256/400
 - 0s - loss: 14.5547
Epoch 257/400
 - 0s - loss: 14.4223
Epoch 258/400
 - 0s - loss: 14.3283
Epoch 259/400
 - 0s - loss: 

/Users/safdharalishaik/opt/anaconda3/lib/python3.7/site-packages/plotly/matplotlylib/mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

127.0.0.1 - - [01/Sep/2020 17:24:25] "POST /_dash-update-component HTTP/1.1" 200 -
